### Code réalisé par l'équipe 8.
### Équipe : Anas Bouhemhem 1953369
###          Bouh Abdillahi 1940646

Laboratoire 2


In [210]:
import numpy as np
import pandas as pd 
import yaml
from yaml.loader import SafeLoader
import random
import math
import matplotlib.pyplot as plt
import operator

import os
from pathlib import Path

# Inclusion du fichier de pathloss du 3GPP (Méthodologie 1)
from f3GPP import *



## Section Lecture des fichiers d'entrés

#### Lecture du fichier cas8.yaml

In [211]:
# import pyyaml module
# Open the file and load the file
def open_cas_yaml() :
    with open('cas8.yaml') as f:
        dataCas8 = yaml.load(f, Loader=SafeLoader)
        print(dataCas8)




#### Lecture du fichier devices_db.yaml

In [212]:
# import pyyaml module
# Open the file and load the file
def open_devices_yaml() :
    with open('devices_db.yaml') as f:
        dataDevices_db = yaml.load(f, Loader=SafeLoader)
        print(dataDevices_db)

## Création de fichier de coordonnées

### Routine de génération de coordonnées de la structure en grille 

In [213]:
def fill_up_the_lattice(N, lh, lv, nh, nv):
    """Function appelée par get_rectangle_lattice_coords()"""
    
    def get_delta1d(L, n):
        return L/(n + 1)
    
    coords = []
    deltav = get_delta1d(lv, nv)
    deltah = get_delta1d(lh, nh)
    line = 1
    y = deltav
    count = 0
    while count < N:
        if count + nh < N:
            x = deltah
            for  i in range(nh):
                # Fill up the horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        else:
            deltah = get_delta1d(lh, N - count)
            x = deltah
            for i in range(N - count):
                # Fill up the last horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        y = y +deltav
    return coords



def get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv):
    """Function appelee par gen_lattice_coords()"""
    
    if Np > N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    elif Np < N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv + 1)
    else:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    return coords


# In[170]:


def gen_lattice_coords(terrain_shape: dict, N: int):
    """Génère un ensemble de N coordonnées placées en grille 
       sur un terrain rectangulaire
    
       Args: terrain_shape: dictionary {'rectangle': {'length' : lh,
                                                   'height' : lv}
           lh and lv are given in the case file"""

    shape = list(terrain_shape.keys())[0]
    lh = terrain_shape[shape]['length']
    lv = terrain_shape[shape]['height']
    R = lv / lh    
    nv = round(math.sqrt(N / R))
    nh = round(R * nv)
    Np = nh * nv
    if shape.lower() == 'rectangle':
        coords = get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv)
    else:
        msg = [f"\tImproper shape ({shape}) used in the\n",
                "\tgeneration of lattice coordinates.\n"
                "\tValid values: ['rectangle']"]
        ERROR(''.join(msg), 2)
    return coords

### Routine des coordonnées en grille

In [214]:
shape= {'rectangle': {'length': 200, 'height': 200}}
N = 32
coords = gen_lattice_coords(shape, N)
print(coords)

[(28.571428571428573, 28.571428571428573), (57.142857142857146, 28.571428571428573), (85.71428571428572, 28.571428571428573), (114.28571428571429, 28.571428571428573), (142.85714285714286, 28.571428571428573), (171.42857142857144, 28.571428571428573), (28.571428571428573, 57.142857142857146), (57.142857142857146, 57.142857142857146), (85.71428571428572, 57.142857142857146), (114.28571428571429, 57.142857142857146), (142.85714285714286, 57.142857142857146), (171.42857142857144, 57.142857142857146), (28.571428571428573, 85.71428571428572), (57.142857142857146, 85.71428571428572), (85.71428571428572, 85.71428571428572), (114.28571428571429, 85.71428571428572), (142.85714285714286, 85.71428571428572), (171.42857142857144, 85.71428571428572), (28.571428571428573, 114.28571428571429), (57.142857142857146, 114.28571428571429), (85.71428571428572, 114.28571428571429), (114.28571428571429, 114.28571428571429), (142.85714285714286, 114.28571428571429), (171.42857142857144, 114.28571428571429), (

### Calcul de Pathloss

In [215]:
def calcul_pathloss(distance_ue_antenne):
    """Calculate the pathloss (substitution for the 3gpp formulas)"""
    if  (distance_ue_antenne == 0) :
        pathloss=500*random.random()
    else : 
        pathloss = distance_ue_antenne
    return pathloss


### Section Lab 2

In [216]:
#pathCoord = os.path.abspath("") + '\\' + "test" 

# Lecture de fichier coordonées si existant:

isExistingCoord = os.path.isfile("coords_lab2_eq8.txt")
print(isExistingCoord)
#if (isExistingCoord == false):
    #création du fichier.


# Define Data : 

print(dataCas8['ETUDE_IMPORTANT']['COORD_FILES'])

if 'write' in dataCas8['ETUDE_IMPORTANT']['COORD_FILES']:
    print("Succes!")
else :
    print("Failure!")
    


False
{'read': 'data/coordonnees_case9.txt'}
Failure!


## Section Antennes/UE


### Objet Antennes et UE (User Equipement)

In [217]:

class Antenna:
    
    def __init__(self, id):
        self.id = id #id de l'antenne (int)
        self.group = None # group défini dans la base de données (str)
        self.coords = None # tuple contenant les coordonnées (x,y) 
        self.assoc_ues = [] # liste avec les id des UEs associés à l'antenne
        self.scenario = None # scénario tel que lu du fichier de cas (str)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
        
    
class UE:
    
    def __init__(self, id, app_name):
        self.id= id #id de l'UE (int)
        self.group = None # group défini dans la base de données (str)
        self.coords=None # tuple contenant les coordonnées (x,y) 
        self.app=app_name #nom de l'application qui tourne dans le UE (str)
        self.assoc_ant=None #id de l'antenne associée à l'UE (int)
        self.los = True # LoS ou non (bool)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
       

### Creation d'objets Antennes/UE et determination des coords

In [218]:
def place_ue_and_antennas():

###################################################
#              Section Antenne   
##################################################

    shape = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']
    global Nb_antenne
    Nb_antenne = dataCas8['ETUDE_IMPORTANT']['DEVICES']['Antenna1']['number'] 
    
    global listAntenne # facon d'acceder globalement a la liste
    listAntenne = []

    for x in range(N):
        listAntenne.append(Antenna(x))
        


    # Generation des coordonnees
    coords_antennes = gen_lattice_coords(shape, N)
  
    for x in range(N):
        listAntenne[x].coords = ([item[0] for item in coords_antennes][x],
                                [item[1] for item in coords_antennes][x])



    #############################
    # Test a des fin pratiques, a effacer apres
    #############################
    
    coords_antennes_x = [item[0] for item in coords_antennes]
    coords_antennes_y = [item[1] for item in coords_antennes]

    


###################################################
#              Section UE   
##################################################

    global nb_ue
    nb_ue =  \
    dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number'] + \
    dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']


    nb_ue_app_1 = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number']
    nb_ue_app_2 = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']
    
    coords_ue_x = []
    coords_ue_y = []
    coords_ue = []

    #*******************************************
    #   Creation des objets UE
    #*******************************************

    global listUE # facon d'acceder globalement a la liste (voir Antenne)
    listUE = []

    #Qte de UE app 1

    for x in range(nb_ue_app_1):
        listUE.append(UE(x, 'UE1-App1'))

    #Qte de UE app 2

    for x in range(nb_ue_app_2):
        listUE.append(UE(nb_ue_app_1 + x, 'UE2-App2'))


    #grid element layout generation and attribution

    for x in range(nb_ue):
        temp = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']['rectangle']['length'] \
            *random.random()
        coords_ue_x.append(temp)
        temp2 = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']['rectangle']['height'] \
            *random.random()
        coords_ue_y.append(temp2)

    #index = 0
    #while index < nb_ue:
    #    coords_ue.append((coords_ue_x[index], coords_ue_y[index]))
    #    index = index + 1


    # Attribution des coords dans les UE

    for x in range(len(listUE)):
        setattr(listUE[x], "coords", (coords_ue_x[x], coords_ue_y[x]))
    print(coords_ue)


    #calcul de distance 
    distance = []

    return listAntenne, listUE




Section de test d'affichage


In [219]:
shape = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']
#N = dataCas8['ETUDE_IMPORTANT']['DEVICES']['Antenna1']['number'] + \
#dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number'] + \
#dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']

#place_ue_and_antennas()
#print(shape)

#test = Antenna(1)
#print(test.coords)

#test2 = UE(3,"test")
#print("test 2 id = ", test2.id)
#print("test 2 group = ", test2.group)
#print("test 2 app name = ", test2.app)

place_ue_and_antennas()

#setattr(listAntenne[6], "coords", (1,2))
print(listUE[6].__dict__)

for x in range(len(listUE)):
    print(listUE[x].coords)    




[]
{'id': 6, 'group': None, 'coords': (358.04823471216383, 3139.1245060038937), 'app': 'UE1-App1', 'assoc_ant': None, 'los': True, 'gen': None}
(167.59905653890905, 231.06590183959287)
(2528.9600031990017, 1489.9280931111773)
(1420.2212216124603, 2079.8595219154627)
(2699.252521230328, 124.7145342123006)
(37.95185769198406, 1062.5762987383234)
(1731.3475048843425, 1219.5272401803154)
(358.04823471216383, 3139.1245060038937)
(3811.710725319345, 1954.7888358521743)
(186.8959530970532, 1454.3212195144201)
(4948.516213891423, 1434.0928021986965)
(1776.5317588787511, 873.8413021800158)
(2595.2678634914405, 257.55140855396564)
(2829.133215407661, 2981.2176968861763)
(4574.473376459784, 1520.3656898966144)
(2447.1776780358155, 2781.8602351367017)
(1940.9835467659664, 1972.7136936015545)
(2251.2201184941546, 1629.5100264217233)
(843.3627446034214, 2538.649718929856)
(1771.1534551416314, 2097.21400246486)
(571.9414188112044, 1842.4305295452093)
(3511.9318040459457, 2563.1026913186042)
(3556.381

In [220]:
#def calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne):

def calcul_association_ue_antenne(pathloss_computation):
     
    index_antennes = 0
    index_ue = 0
    smallestDistance = [0] * nb_ue
    check = 0
    calculPathlossChaqueAntenne = [] * Nb_antenne # nom a changer
    pathlossTotal = [] * nb_ue 

    temporary_value = 0
    for x in range(nb_ue):
        for y in range(Nb_antenne):
            temp = tuple(map(operator.sub, listUE[x].coords, listAntenne[y].coords))
            # print("HERE ARE COORDS OF LISTUE AND LISTANTENNE : ", listUE[x].coords, listAntenne[y].coords)

            if pathloss_computation == "random" :
                check = calcul_pathloss(0) #random pathloss
                calculPathlossChaqueAntenne.append(calcul_pathloss(0))                

            if pathloss_computation == "distance" :
                check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
                print(check)
                calculPathlossChaqueAntenne.append(check)
                
            if y == 0: # Assigner premiere valeur
                path_loss_min = check 
                smallestDistance[x] = (check, y) #antenne commence a 1, et non a 0.
                # print("Here is pathloss min : ", path_loss_min)
                # print("Here is check : ", check)
                # print("Here is y : ", y)
            elif check < path_loss_min: #sinon comparer pour la plus petite
                path_loss_min = check
                smallestDistance[x] = (check, y) #antenne commence a 1, et non a 0.
                temporary_value = y
                # print("NOUS SOMMES DANS LE ELIF !!! \n")
                # print("Here is pathloss min : ", path_loss_min)
                # print("Here is check : ", check)
                # print("Here is y : ", y)
        # path_loss_min = 1000000000000
        #print(calculPathlossChaqueAntenne)
        pathlossTotal.append(calculPathlossChaqueAntenne)
        calculPathlossChaqueAntenne = [] * Nb_antenne # nom a changer
        setattr(listUE[x], "assoc_ant", (temporary_value))

    
    # print(pathlossTotal)

    return smallestDistance, pathlossTotal   



def set_assoc_ues():
    temp = []
    for ue in listUE :
        temp.append(ue.assoc_ant)
        # listAntenne[temp]

        # setattr(listAntenne[temp], "assoc_ues", (temporary_value))
    
    print(temp) 
    id_antenne_ue = []   
    # for ue in listUE:
    for ant_number in range(Nb_antenne) :
        # print(ant_number)
        for z in range(nb_ue) :
            if (temp[z] == ant_number) :
                id_antenne_ue.append(z) #append numero ue
        print("here is list of ue for antenna number " , ant_number, " : ", id_antenne_ue)
        setattr(listAntenne[ant_number], "assoc_ues", (id_antenne_ue))
        id_antenne_ue = []


# set_assoc_ues()
# print(listAntenne[4].__dict__)
    


### Section Test 

In [221]:
smallestDistance, pathlossTotal = calcul_association_ue_antenne("distance")
print(smallestDistance)
print(listUE[5].__dict__)

718.0025601990195
1523.0000817096166
2347.8542778910996
3177.136928183057
4008.1002107639833
1017.0849397236957
1684.7739750625024
2455.881472785677
3257.7804036774605
4072.3213770141833
1432.9675058329692
1964.0258260033934
2655.237824853426
3410.5816595919946
4195.561404132599
1890.0608378921477
2318.691774119346
2927.323351577813
3626.430938940612
4372.833131278566
2364.5853905270533
2719.4237701265183
3253.9446988006985
3894.8601326705043
4597.891222307017
1963.4427501433465
1312.824218149101
990.3516099426596
1275.5289427379241
1913.6459299194103
1764.9870651025435
991.7556829419335
490.78327009469325
941.8311901030615
1709.4188053212551
1695.6565828221565
862.3521563354751
30.661459418352894
804.4363850173436
1637.7376343220456
1770.6843744035625
1001.8598447014764
510.8933665477425
952.4651197963076
1715.3006925211662
1973.6755206076195
1328.0794612198808
1010.486981049884
1291.2248879716567
1924.143542631998
1685.346700947816
1598.9657489686638
1913.6034906017082
2481.119517677

### Écriture dans un fichier

In [222]:
def write_file_assoc(pathloss_computation, pl_lab1_rand_eqn, assoc_lab1_rand_eqn, coords_ue, coords_antennes, N_ue, N_antenne):
    return

    # id_ue, numerant, pathloss = calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)


    # data = {"ID UE": id_ue, "ID antenne": numerant,
    # "Pathloss entre UE et antenne": pathloss
    # }

    # id_ant, numerue, pathloss = inv_calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)
    # data_inv = {"ID antenne": id_ant, "ID UE": numerue, 
    # "Pathloss entre UE et antenne": pathloss
    # }

    # df = pd.DataFrame(data=data)
    # df_inv = pd.DataFrame(data=data_inv)

    # grp = df.sort_values(by=['ID antenne'])
    # new_df = grp['ID antenne'].tolist()
    # new_df_id_ue = grp['ID UE'].tolist()
    # list_antenne_sorted = list(range(N_antenne + 1))
    # list_antenne_sorted = list_antenne_sorted[1:]
    # buffer = []
    # vector = [0]*N_antenne
    # current_ue_selection = ""

    # #This let's us associate an antenna to a list of specified UEs
    # temp = 0
    # for j in  id_ue :
    #     if temp == 0 and j == 0 :
    #         current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1] ) + " "
    #     if j < 300 and j > 0 :
    #         if new_df[j-1] == new_df[j] and j < 300 and j > 0 :
    #             current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1]) + " "
    #         else :
    #             buffer.append(current_ue_selection)
    #             vector[temp] = buffer 
    #             buffer = []
    #             current_ue_selection = ""
    #             temp = temp + 1

    #     if temp == (N_antenne-1) :
    #         if (j == N_ue):
    #             current_ue_selection = current_ue_selection + " " + str(new_df_id_ue[j-1])
    #             buffer.append(current_ue_selection)
    #             vector[temp] = buffer 

    # #Dataframe to encapsulate the association antenna to list of UEs
    # data_assoc = { "ID antenne": list_antenne_sorted, "List ID UE": vector
    # }
    # df_assoc = pd.DataFrame(data=data_assoc)
    
    #Writing necessary files
    # path = os.path.abspath("") + '\\' + pl_lab1_rand_eqn 
    # path_inv = os.path.abspath("") + '\\' + assoc_lab1_rand_eqn 
    # with open(path, 'w') as f:
    #     df_string = df.to_string(header=False, index=False)
    #     f.write(df_string)
    # f.close()
    # with open(path_inv, 'w') as f_inv:
    #     df_string_inv = df_assoc.to_string(header=False, index=False)
    #     f_inv.write(df_string_inv)
    # f_inv.close()



### Fonction lab1_eq8


In [238]:
def lab1_eq8() :
    # N_ue = 300
    # N_antenne = 25
    open_cas_yaml()
    open_devices_yaml()
    listAntenne, listUE = place_ue_and_antennas()


    user_choose_name = input("Voulez-vous choisir le nom des fichiers à ecrire? ")   
    user_choose_pl_computation = input("Comment calculer pathloss? (par defaut proportionnel a distance)")   

    if (user_choose_pl_computation == "random") :
        #pathloss random
        if user_choose_name == "oui" :
            pl_lab1_eqn = input("Quel nom de fichier pour pl_lab1_rand_eqn.txt ")
            assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab1_rand_eqn.txt ")
        else :
            pl_lab1_eqn = "pl_lab1_rand_eq8.txt"
            assoc_lab1_eqn = "assoc_lab1_rand_eq8.txt" 

        pathloss_computation = "random"
        # write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)    
        calcul_association_ue_antenne(pathloss_computation)
        set_assoc_ues()
        print(listAntenne[4].__dict__)
    
    else :
        #pathloss proportional to distance
        if user_choose_name == "oui" :
            pl_lab1_eqn = input("Quel nom de fichier pour pl_lab1_dist_eqn.txt ")
            assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab1_dist_eqn.txt ")
        else :
            pl_lab1_eqn = "pl_lab1_dist_eq8.txt"
            assoc_lab1_eqn ="assoc_lab1_dist_eq8.txt"

        pathloss_computation = "distance"
        # write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)    
        calcul_association_ue_antenne(pathloss_computation)
        set_assoc_ues()
        # print(listAntenne[4].__dict__)


    return listAntenne, listUE


### Fonction lab1()

In [239]:
def lab1() :
    listAntenne, listUE = lab1_eq8()
    for x in range(nb_ue) :
        print("UE numero ", listUE[x].id, " vaut : ", listUE[x].__dict__)
    for y in range(Nb_antenne) :
        print("Antenne numero ", listAntenne[y].id, " vaut : ", listAntenne[y].__dict__)
    # return listAntenne, listUE
    

### Fonction Main

In [241]:
lab1()

{'ETUDE_IMPORTANT': {'SCENARIO': 'Uma', 'ANT_COORD_GEN': 'g', 'UE_COORD_GEN': 'a', 'COORD_FILES': {'read': 'data/coordonnees_case8.txt'}, 'DEVICES': {'Antenna1': {'number': 25}, 'UE1-App1': {'number': 100}, 'UE2-App2': {'number': 200}}, 'GEOMETRY': {'Surface': {'rectangle': {'length': 1000, 'height': 500}}}}}
{'ANTENNAS': {'Antenna1': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 40}, 'Antenna2': {'type': 'antenna', 'name': 'Antenna2', 'height': 35, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}, 'Antenna3': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 45}, 'Antenna4': {'type': 'antenna', 'name': 'Antenna2', 'height': 37, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}}, 'UES': {'UE1-App1': {'type': 'ue', 'name': 'UE1-App1', 'app': 'app1', 'height': 1.5}, 'UE2-App2': {'type': 'ue', 'name': 'UE2-App2', 'app': 'app2', 'height': 1.5}}}
[]
[6, 4, 14, 17, 2, 19, 8, 1, 22, 15, 20, 4, 7

## Traitement de scénario de propagation 3GPP (UMi, UMa, etc.)

Le module python qui fait les calculs de pathloss du 3GPP. Veillez l'appeler ```calcul_pathloss_3gpp_eqn.py```

## Calcul de pathloss basés sur ces scénarios
Faire des calculs de pathloss basés sur ces scénarios


## Association des UEs pour chaque antenne
Pour chaque UE, montrer l'antenne associée

## Montrer la liste des UEs associés pour CHAQUE Antenne
Pour chaque antenne, montrer la liste des UEs qu'y sont associés.

## Fournir les fichiers suivants :


    Le fichier de cas utilisé, appelé ```cas_lab2_eqn.yaml```
    Le fichier de coordonnées, appelé ```coords_lab2_eqn.txt``` (voir format dans section méthodologie)
    Le fichier de visibilité, appelé ```visibility_lab2_eqn.txt``` (voir format dans exemple ci-dessus)
    Un fichier d'association, appelé ```assoc_ant_lab2_eqn.txt``` qui montre pour chaque antenne, la liste des UEs associées (voir format dans livrables Lab1)
    Un fichier d'association, appelé ```assoc_ue_lab2_eqn.txt``` qui montre pour chaque UE, l'antenne associé
    Le fichier de pathloss, appelé ```pl_lab2_eqn.txt```  qui montre le pathloss pour chaque UE et antenne  (voir format dans livrables Lab1)